In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np
from tqdm import tqdm

In [2]:
# Get configuration file that has all the api tokens
def configure_undoc():
    load_dotenv()
    UNDOC_AUTHORIZATION = (os.getenv('UNDOC_AUTHORIZATION'))
    UNDOC_CLIENT_TOKEN = (os.getenv('UNDOC_CLIENT_TOKEN'))
    return UNDOC_AUTHORIZATION, UNDOC_CLIENT_TOKEN

In [3]:
#Grab the authorization and client token from the env
UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN = configure_undoc()
print(UNDOC_AUTHORIZATION)
print(UNDOC_CLIENT_TOKEN)

BQC-kyIhZ0UNnLRacHfaqFMuUFQieBODDA7Qtrj-JgnhLIoxYQB37qZbpUVJ9bKuQPIZoQU0ihMSo-a2uMQ_bGwkz8H2bQrheMI2Z07h-rVpHrYPA3O0cNIjLjK9Hdb4lQVVfCzAx_kvEoV23C0XPtFLJKRFJpoZgNZZt-mSsO2tq5CbGDFgBNiHLTsPSovngZzxATgV80h3jfGhfTvzZ6swyGXWpT1pWVL31bn3xK6DPGtGIX9E8_DQhYED3NeJGxPHENmWoQEv_7ORcvLlUw1pAVit7mawJPR4WLOSCMcMHMhECJll1SlPM0xdztwqZ8Mmxw
AABgim22BLnPeO3Gha9Me2MaGuboabqA2m2wdKxHzunrtwEY+0hbPIMRCbM07+odJ4c8v0y0hLbRSb51mGB+iDVGFp/3/nMv1k/fIWF8UVPFcG3724kZD6G42zXuixL9/w6Nsj0TNsva35qzO8qM7dAYhuGjCSD9KzfPH5hRxVKulyRUZk3wnD24ppPT1O+hkMivDmpgAgRDhaei4osjt01fNsV8eDoPxU2r/pv10YU9kuxJ2V7bfXicGWaGaQ/vM665+rH8NC6BOsZ6d/XXWTrdrxoNtjLlvTwDeTNGUA==


***

## Get the play counts of each song in the Album

#### `request_album_playcounts()`

In [4]:
def request_album_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,album_ID):
    '''Search ID of albums and return response to get playcounts and track_uri'''
    url = "https://api-partner.spotify.com/pathfinder/v1/query"

    querystring = {
                "operationName":"queryAlbumTracks",
                "variables":'{\"uri\":\"spotify:album:' +album_ID+ '\",\"offset\":0,\"limit\":300}',
                "extensions":"{\"persistedQuery\":{\"version\":1,\"sha256Hash\":\"f387592b8a1d259b833237a51ed9b23d7d8ac83da78c6f4be3e6a08edef83d5b\"}}"
                }

    headers = {
    "authority": "api-partner.spotify.com",
    "accept": "application/json",
    "accept-language": "en",
    "app-platform": "WebPlayer",
    "authorization":"Bearer {}".format(UNDOC_AUTHORIZATION),
    "client-token": str(UNDOC_CLIENT_TOKEN),
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://open.spotify.com",
    "referer": "https://open.spotify.com/"
}

    #prcoessing the result
    result = requests.request("GET", url, headers=headers, params=querystring)
    json_result = result.json()
    #getting each track object
    items = json_result['data']['albumUnion']['tracks']['items']
    
    # get the number of items to determine the loop size
    n = len(items)
    track_uri_list = np.empty(n, dtype=object)
    playcount_list = np.empty(n)
    for i in range(n):
        track_uri_list[i] = items[i]['track']['uri']
        playcount_list[i] = items[i]['track']['playcount']

    return track_uri_list, playcount_list

Test what `request_album_playcounts()` return

In [5]:
request_album_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,'5MS3MvWHJ3lOZPLiMxzOU6')

(array(['spotify:track:1bDbXMyjaUIooNwFE9wn0N',
        'spotify:track:46s57QULU02Voy0Kup6UEb',
        'spotify:track:34tz0eDhGuFErIuW3q4mPX',
        'spotify:track:0wshkEEcJUQU33RSRBb5dv',
        'spotify:track:7l2nxyx7IkBX5orhkALg0V',
        'spotify:track:2ZL7WZcjuYKi1KUDtp4kCC',
        'spotify:track:0sSRLXxknVTQDStgU1NqpY',
        'spotify:track:4Flfb4fGscN9kXPOduQLrv',
        'spotify:track:7GeTsDIc5ykNB6lORO6Cee',
        'spotify:track:2KLwPaRDOB87XOYAT2fgxh',
        'spotify:track:45pUWUeEWGoSdH6UF162G8',
        'spotify:track:410TZrK18uRjtsTunG14cl',
        'spotify:track:2ZlABhxMMa43COmZ97kKsJ',
        'spotify:track:29YW2xXlo6Pc6M0SBTCXYN',
        'spotify:track:1MTXYvJ9TSqg9x6WPCDx2n',
        'spotify:track:31bsuKDOzFGzBAoXxtnAJm'], dtype=object),
 array([3.81681747e+08, 1.20395783e+08, 1.16635003e+08, 7.13407670e+07,
        8.03696400e+07, 1.05473437e+08, 6.09217450e+07, 4.79212130e+07,
        1.01589261e+08, 1.39488978e+08, 7.76550460e+07, 4.51793480e+07,


#### `get_today_playcounts()`

In [6]:
def get_today_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,album_ID):
    '''Using the undocumented API to request playcounts and return a dataframe of '''
    track_uri_list, playcount_list = request_album_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,album_ID)
    df = pd.DataFrame({'track_uri':track_uri_list, 'playcount':playcount_list})
    df['date_tracked'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df

Test what `get_today_playcounts()`

In [7]:
get_today_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,"5MS3MvWHJ3lOZPLiMxzOU6")

,track_uri,playcount,date_tracked
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,381681747.0,2023-03-04
1,spotify:track:46s57QULU02Voy0Kup6UEb,120395783.0,2023-03-04
2,spotify:track:34tz0eDhGuFErIuW3q4mPX,116635003.0,2023-03-04
3,spotify:track:0wshkEEcJUQU33RSRBb5dv,71340767.0,2023-03-04
4,spotify:track:7l2nxyx7IkBX5orhkALg0V,80369640.0,2023-03-04
5,spotify:track:2ZL7WZcjuYKi1KUDtp4kCC,105473437.0,2023-03-04
6,spotify:track:0sSRLXxknVTQDStgU1NqpY,60921745.0,2023-03-04
7,spotify:track:4Flfb4fGscN9kXPOduQLrv,47921213.0,2023-03-04
8,spotify:track:7GeTsDIc5ykNB6lORO6Cee,101589261.0,2023-03-04
9,spotify:track:2KLwPaRDOB87XOYAT2fgxh,139488978.0,2023-03-04


#### `get_all_playcounts_today()`
**Being used in update_track_playcount**

In [8]:
def get_all_playcounts_today(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,albums_ids):
    '''Takes in array of album_ids and loop through each one to get a dataframe to be able to append to SQL database'''
    dataframe = pd.DataFrame()
    for id in tqdm(albums_ids, desc = "Updating track playcounts"):
        temp = get_today_playcounts(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,id)
        dataframe = pd.concat([dataframe,temp], axis = 0)
    dataframe  = dataframe.reset_index(drop=True)
    return dataframe

Test what `get_all_playcounts_today()` return

In [9]:
test_list = np.array(['5MS3MvWHJ3lOZPLiMxzOU6', '3cf4iSSKd8ffTncbtKljXw','3SpBlxme9WbeQdI9kx7KAV', '6OQ9gBfg5EXeNAEwGSs6jK'], dtype=object)

In [10]:
get_all_playcounts_today(UNDOC_AUTHORIZATION,UNDOC_CLIENT_TOKEN,test_list)

Updating track playcounts: 100%|██████████| 4/4 [00:00<00:00,  7.66it/s]


,track_uri,playcount,date_tracked
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,381681747.0,2023-03-04
1,spotify:track:46s57QULU02Voy0Kup6UEb,120395783.0,2023-03-04
2,spotify:track:34tz0eDhGuFErIuW3q4mPX,116635003.0,2023-03-04
3,spotify:track:0wshkEEcJUQU33RSRBb5dv,71340767.0,2023-03-04
4,spotify:track:7l2nxyx7IkBX5orhkALg0V,80369640.0,2023-03-04
...,...,...,...
60,spotify:track:6Kj17Afjo1OKJYpf5VzCeo,144391033.0,2023-03-04
61,spotify:track:6fLVTVaHWaEfVKfEgbkf4D,31909288.0,2023-03-04
62,spotify:track:0YkUwXxnTkeJBvt5upeEtP,58751747.0,2023-03-04
63,spotify:track:05aZ9sAU1YXndHv0FMi9iW,74030238.0,2023-03-04
